In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [3]:
answers = {}  # создадим словарь для ответов
data['profit'] = data['revenue'] - data['budget']  # прибыль от проката
data['release_date'] = data['release_date'].apply(
    pd.to_datetime)  # форматируем дату выпуска в формат datetime
data['release_month'] = data['release_date'].apply(
    lambda x: x.month) # будет удобнее работать со столбцом месяцев в некоторых вопросах
data['is_high_budget'] = data['budget'] > data['budget'].mean()
# фрейм data_to_explode мы будем растаскивать на строки по отдельным жанрам\режиссёрам\актёрам\студиям
# он сразу не взорван по всем столбцам намеренно, см. ниже
data_to_explode = data.copy()
data_to_explode['cast'] = data_to_explode['cast'].apply(
    str.split, args='|')  # разбиваем актёров на отдельные элементы списка
data_to_explode['genres'] = data_to_explode['genres'].apply(
    str.split, args='|')  # разбиваем жанры
data_to_explode['production_companies'] = data_to_explode['production_companies'].apply(
    str.split, args='|')  # разбиваем студии
data_to_explode['director'] = data_to_explode['director'].apply(
    str.split, args='|')

Проверим как растёт объём датасета при взрывании всех ячеек

In [4]:
data.info(memory_usage='deep') #оригинальный

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

In [42]:
exploded = (data_to_explode.explode('cast')
                           .explode('genres')
                           .explode('production_companies')
                           .explode('director'))
exploded.info(memory_usage='deep') # взорвали всё
del exploded

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89552 entries, 0 to 1888
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               89552 non-null  object        
 1   budget                89552 non-null  int64         
 2   revenue               89552 non-null  int64         
 3   original_title        89552 non-null  object        
 4   cast                  89552 non-null  object        
 5   director              89552 non-null  object        
 6   tagline               89552 non-null  object        
 7   overview              89552 non-null  object        
 8   runtime               89552 non-null  int64         
 9   genres                89552 non-null  object        
 10  production_companies  89552 non-null  object        
 11  release_date          89552 non-null  datetime64[ns]
 12  vote_average          89552 non-null  float64       
 13  release_year     

Объём датасета вырос в 44 раза! Для изначального объёма в 2 МБ это неважно,  
но если бы он был в 200 МБ, то уже не влез бы в оперативную память.  
Поэтому я предпочту другой путь - взрывать по мере необходимости и удалять ненужные взорванные фреймы.

# 1. У какого фильма из списка самый большой бюджет?

Вариант 1

In [54]:
answers['1'] = (data[data['budget'] == data['budget'].max()]
                ['original_title'].iloc[0])
answers['1']

'Pirates of the Caribbean: On Stranger Tides'

#### Вариант 2

In [56]:
answers['1'] = data.query('budget == budget.max()')['original_title'].iloc[0]
# query лучше читаемо чем при выделении ячеек по условию
# дальше я буду пользоваться почти только этой функцией для выбора по условию
answers['1']

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
answers['2'] = (data.query('runtime == runtime.max()')
                ['original_title'].iloc[0])
answers['2']

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?

In [9]:
answers['3'] = (data.query('runtime == runtime.min()')
                ['original_title'].iloc[0])
answers['3']

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [10]:
answers['4'] = data['runtime'].mean()
answers['4']

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [11]:
answers['5'] = data['runtime'].median()
answers['5']

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [12]:
answers['6'] = data.query('profit == profit.max()')['original_title'].iloc[0]
answers['6']

'Avatar'

# 7. Какой фильм самый убыточный? 

In [13]:
answers['7'] = data.query('profit == profit.min()')['original_title'].iloc[0]
answers['7']

'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [14]:
answers['8'] = data[data['profit'] > 0]['imdb_id'].count()
answers['8']

1478

#### Вариант 2

In [15]:
len(data.query('profit>0'))

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [16]:
data_2008 = data[data['release_year'] == 2008]
# можно обойтись и без создания отдельного фрейма, он тут для лаконичности записи
answers['9'] = (data_2008.query('profit == profit.max()')
                ['original_title'].iloc[0])
answers['9']

'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [17]:
data_12_14 = data[data['release_year'].isin((2012, 2013, 2014))]
answers['10'] = (data_12_14.query('profit == profit.min()')
                 ['original_title'].iloc[0])
answers['10']

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [18]:
genres = Counter(data_to_explode['genres'].sum())
# sum просто склеит все списки жанров подряд из колонки genres
answers['11'] = genres.most_common(1)[0][0]
answers['11']

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

#### Вариант 1 (умный)

In [19]:
answers['12'] = (data_to_explode.explode('genres')
                 [['imdb_id', 'genres', 'profit']]
                 .query('profit > 0')
                 .groupby(by=['genres']).count()
                 ['imdb_id'].idxmax())
answers['12']

'Drama'

#### Вариант 2 (не очень умный)
Таким методом можно решить почти все следующие вопросы.  
Все такие решения собраны в файле Movies_IMBD_v4.1_stupid_version.ipynb

In [20]:
genres = Counter(data['genres'].apply(str.split, args=('|')).sum())
genre_profitable = Counter()
for genre in genres:
    genre_profitable.update({genre: data[data['genres'].str.contains(genre) 
                                         & data['profit'] > 0]
                                         ['imdb_id'].nunique()})
answers['12'] = genre_profitable.most_common(1)[0][0]
answers['12']

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [21]:
directors_exploded = (data_to_explode.explode('director')
                      [['imdb_id', 'original_title', 
                        'revenue', 'director', 'genres']])
answers['13'] = (directors_exploded.groupby('director').sum()
                                    ['revenue'].idxmax())
answers['13']

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [22]:
answers['14'] = (directors_exploded.explode('genres')
                 .groupby(by=['genres', 'director'])['imdb_id'].count()
                 .loc['Action'].idxmax())
del directors_exploded
answers['14']

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [23]:
actors_exploded = data_to_explode.explode('cast')
answers['15'] = (actors_exploded.query('release_year == 2012')
                 .groupby(by=['cast'])['revenue'].sum()
                 .idxmax())
answers['15']

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [24]:
answers['16'] = (actors_exploded.query('budget > budget.mean()')
                                .groupby('cast')['imdb_id'].count()
                                .idxmax())
answers['16']

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

#### Вариант 1 через groupby

In [25]:
answers['17'] = (actors_exploded.explode('genres')
                                .query('cast == "Nicolas Cage"')
                                [['imdb_id', 'cast', 'genres']]
                                .groupby(by = 'genres').count()
                                ['imdb_id'].idxmax())
answers['17']

'Action'

#### Вариант 2 через pivot

In [26]:
answers['17'] = (actors_exploded.explode('genres')
                                [['imdb_id', 'cast', 'genres']]
                                .pivot_table(index = ['cast'], 
                                             columns = 'genres', 
                                             aggfunc = 'count',
                                             fill_value = 0)
                                .loc['Nicolas Cage'].idxmax()[1])
del actors_exploded # освободим память, больше не понадобятся
answers['17']

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [27]:
studios_expl = (data_to_explode.explode('production_companies')
                              [['imdb_id', 'production_companies', 'release_year',
                                'revenue', 'profit', 'original_title', 'overview']])
answers['18'] = (studios_expl[studios_expl['production_companies']
                              .str.contains("Paramount Pictures")]
                              .query('profit == profit.min()')
                              ['original_title'].iloc[0])
# query очень плохо дружит со строками и по сути не умеет искать подстроку в строке
# эта особенность будет очень актуальна на 20 вопросе
answers['18']

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [28]:
answers['19'] = (data.groupby(by='release_year').sum()
                    .query('revenue == revenue.max()')
                    .index[0])
answers['19']

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

На самом деле в датасете есть много студий с Warner Bros в названии:
Warner Bros.  
Warner Bros. Animation  
Warner Bros. Interactive Entertainment  
Warner Bros. Pictures  
Я буду считать, что нужна информация **по сумме всех их.**


#### Вариант 1

In [29]:
answers['20'] = (studios_expl[studios_expl['production_companies']
                              .str.contains("Warner Bros")]
                              .groupby(by='release_year').sum()
                              ['profit'].idxmax())
answers['20']

2014

#### Вариант 1 через pivot_table

In [30]:
yearly_profit = (studios_expl.pivot_table(values = 'profit', 
                                         index = 'production_companies', 
                                         columns = 'release_year', 
                                         aggfunc = 'sum', 
                                         fill_value = 0))
answers['20'] = (yearly_profit[yearly_profit.index.str.contains("Warner Bros")]
                             .sum().idxmax())
answers['20']


2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [31]:
month_count = data.groupby(by='release_month').nunique()['imdb_id']
answers['21'] = month_count.idxmax()
answers['21']

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [32]:
answers['22'] = (month_count[6] + month_count[7] + month_count[8])
answers['22']

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [33]:
answers['23'] = (data.explode('director')
                     .query('release_month in [1,2,12]')
                     [['director', 'imdb_id']]
                     .groupby('director')
                     ['imdb_id'].count()
                     .idxmax())
answers['23']

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [34]:
studios_expl['title_len'] = studios_expl['original_title'].apply(len)
answers['24'] = (studios_expl.groupby('production_companies').mean()
                             .query('title_len == title_len.max()')
                             .index[0])
answers['24']

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [35]:
studios_expl['overview_len'] = studios_expl['overview'].apply(len)
answers['25'] = (studios_expl.groupby('production_companies').mean()
                             .query('overview_len == overview_len.max()')
                             .index[0])
del studios_expl # освободим память
answers['25']

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

Вот список всех таких фильмов

In [36]:
data.query('vote_average > vote_average.quantile(0.99)')[['original_title', 'vote_average']]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


Очень лень было сверять вручную, написал автоматическую сверку из вариантов ответа:

In [37]:
top_1_percent = set(data.query('vote_average > vote_average.quantile(0.99)')
                        ['original_title'])
a = 'Inside Out, The Dark Knight, 12 Years a Slave'
b = 'BloodRayne, The Adventures of Rocky & Bullwinkle'
c = 'Batman Begins, The Lord of the Rings: The Return of the King, Upside Down'
d = '300, Lucky Number Slevin, Kill Bill: Vol. 1'
e = 'Upside Down, Inside Out, Iron Man'
print(a, '_______________________', set(a.split(', ')) <= top_1_percent)
print(b, '_______________________', set(b.split(', ')) <= top_1_percent)
print(c, '_______________________', set(c.split(', ')) <= top_1_percent)
print(d, '_______________________', set(d.split(', ')) <= top_1_percent)
print(e, '_______________________', set(e.split(', ')) <= top_1_percent)

Inside Out, The Dark Knight, 12 Years a Slave _______________________ True
BloodRayne, The Adventures of Rocky & Bullwinkle _______________________ False
Batman Begins, The Lord of the Rings: The Return of the King, Upside Down _______________________ False
300, Lucky Number Slevin, Kill Bill: Vol. 1 _______________________ False
Upside Down, Inside Out, Iron Man _______________________ False


In [38]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [39]:
cast_split = data['cast'].apply(str.split, args=('|'))
duets = cast_split.apply(lambda x: list(combinations(x, 2)))
duets_unique = Counter([' & '.join(sorted(list(x))) for x in duets.sum()])
answers['27'] = duets_unique.most_common(1)[0][0]
answers['27']

'Daniel Radcliffe & Rupert Grint'

# Submission

In [40]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 109.6585494970884,
 '5': 107.0,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [41]:
# и убедиться что ни чего не пропустил)
len(answers)

27